初始化环境：
- 导包
- 加载环境变量
- 创建LLM对象

In [ ]:
from dotenv import load_dotenv
import os
from DeepSeekLLM import DeepSeekLLM

load_dotenv()
llm = DeepSeekLLM(api_key=os.getenv("deepseek_api_key"))

测试llm是否初始化成功

In [ ]:
response = llm.invoke("你好！")
print(response)

在 LangChain Chain 中使用

In [7]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["question"],
    template="回答以下问题：{question}"
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.invoke("你是谁？"))

<ipython-input-7-6172cb838d32>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'question': '你是谁？', 'text': '我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的智能AI助手！🤖✨  \n\n我可以帮你解答各种问题，比如学习、工作、编程、写作、生活小窍门等。无论是严肃的知识探讨，还是轻松的日常闲聊，我都会尽力提供有用的信息。  \n\n有什么我可以帮你的吗？😊'}
